# An inner parallel cutting plane method for convex mixed-integer optimization problems 

## Summary of the document

This document is complementary to the paper "Granularity in mixed-integer nonlinear optimization". The intention is to make the numerical results transparent. The generated results for the IPCP, the feasibility pump as well as for B-Hyb are stored as pandas data-frames and are concatenated herein. 

The latex-tables of the article are produced by using the command print(dataframe.to_latex(float_format = '%.2f'))

In [4]:
import pandas as pd
import numpy as np
import pickle
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

## First quick check

In [5]:
result = load_obj('intermediate_results')

In [6]:
result

,vars,constrs,time L,time SOR,obj,constr_value
cvxnonsep_normcon20r,40(10),21(20),2.015106678009033,0.07811188697814941,-14.6549272394,-inf
cvxnonsep_normcon30r,60(15),31(30),1.9681425094604492,0.0781407356262207,-14.8031228018,-inf
ex4,36(25),30(25),0,0.07814168930053711,-6.70152492653,-inf
ex1223a,7(4),9(4),0,0.0781400203704834,6.06978727803,-inf
ringpack_10_1,70(50),385(330),0,0.6116049289703369,-4.17164353115092,-inf
smallinvDAXr1b150-165,31(30),4(1),0.18745183944702148,0.062482595443725586,224.691303163,-124.027294155
